## Concatenating last 4 attention distributions using a GRU - Including Weak Labels

In this experiment the attention distributions we extract are from a BERT model that is *not* pretrained on detecting bias

We concatenate together the last 4 attention distributions of the input. Our classification model is an LSTM that reads in the attention distribution for each word sequentially. In general the max sequence lenght is 80 which means that our model will read in 80 data points which are each 80 dimensional. 


#### Notes
* One remaining question is how can we experiment with different number of attention heads. In general the extract attention scores function seems to have some bugs that need ironing out - such as only being able to pass in a batch size of 1 into the extraction schema.

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist
from src.utils.classification_utils import run_bootstrapping

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
params = Params.read_params("experiment_params.json")

In [4]:
# Dev dataset
dev_dataset = ExperimentDataset.init_dataset(params.dataset)

03/13/2020 11:29:34 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
100it [00:00, 3750.71it/s]


In [5]:
# Loading in weak labels
import pickle 
train_dataset = pickle.load(open(params.dataset["weakly_labeled_data"], "rb"))

In [6]:
train_dataset

Length: 52275 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'pos_features', 'marta_features', 'bert_embeddings', 'weak_bias_label'])

Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [7]:
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True, from_pretrained=False)

03/13/2020 11:29:38 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


The len of our vocabulary is 30523
Cuda is set to true


03/13/2020 11:29:39 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/13/2020 11:29:39 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpqsmf7ay_
03/13/2020 11:29:42 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

03/13/2020 11:29:51 - INFO - pytor

Instantiated joint model with default HuggingFace weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [8]:
attention_dataloader_dev = dev_dataset.return_dataloader() 
attention_dataloader_train = train_dataset.return_dataloader()

In [9]:
attention_scores_dev = attention_experiment.extract_attention_scores(attention_dataloader_dev)

In [10]:
attention_scores_train = attention_experiment.extract_attention_scores(attention_dataloader_train)

In [21]:
import pickle

In [ ]:
pickle.dump(attention_scores_train, open("attention_scores_train.pkl", "wb"))

In [19]:
reduced_attention_train = torch.stack(reduce_attention_dist(attention_scores_train[:1000], params.intermediary_task["attention"]["reducer"])).squeeze()

In [20]:
reduced_attention_train.shape

torch.Size([1000, 80, 320])

In [1]:
reduced_attention_dev = reduce_attention_dist(attention_scores_dev, params.intermediary_task["attention"]["reducer"])
reduced_attention_train = reduce_attention_dist(attention_scores_train, params.intermediary_task["attention"]["reducer"])

stacked_reduced_attention_dev = torch.stack(reduced_attention_dev).squeeze()
stacked_reduced_attention_train = torch.stack(reduced_attention_train).squeeze()
dev_dataset.add_data(stacked_reduced_attention_dev, "attention_dist")
train_dataset.add_data(stacked_reduced_attention_train, "attention_dist")
dev_dataset.shuffle_data()
train_dataset.shuffle_data()

NameError: name 'reduce_attention_dist' is not defined

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [ ]:
train_dataloader = train_dataset.return_dataloader(batch_size=params.final_task['training_params']['batch_size'])
dev_dataloader = dev_dataset.return_dataloader(batch_size=params.final_task['training_params']['batch_size'])

In [8]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [9]:
losses, evals = classification_experiment.train_model(train_dataloader, dev_dataloader, input_key="attention_dist", label_key="weak_bias_label")

NameError: name 'classification_experiment' is not defined

In [ ]:
from src.utils.classification_utils import average_data

In [ ]:
avg_evaluations = [average_data(epoch_evaluations) for epoch_evaluations in evaluations] #averaging evaluations 